In [1]:
import pandas as pd
import requests as rq
import numpy as np
from bs4 import BeautifulSoup as bsp
import time
import re

In [2]:
!curl  http://www.mtime.com/robots.txt






















Crawl-delay: 60

In [3]:
## get the rank and link to each individual movie for top 100 sales in China
for area in ["china","global","NorthAmerica"]:
    with open('mtime_data_'+area+'.txt', mode='w') as file:
        rank=1
        for page in range(10):
            parameters={"area":area,
                       "type":"MovieRankingHistory",
                       "category":"all",
                       "page":f'{page}',
                       "display":"list",
                       "timestamp":f'{time.time()}',
                       "version":"07bb781100018dd58eafc3b35d42686804c6df8d",
                       "dataType":"json"}
            try:
                MtimeRank = rq.get("http://movie.mtime.com/boxoffice/",params=parameters,timeout=10)
            except:
                print(f'Page {page} of area {area} for the MTime Boxoffice connection timeout')
                continue
                
            if MtimeRank.status_code!=200:
                print(f'Page {page} of area {area} for the MTime Boxoffice failed connection.')
                continue

            soup=bsp(MtimeRank.json()['html'])
            basic_info=soup.find_all("div",{"class":"txtbox"})
        
            for info in basic_info:
                for i in info.contents:
                    if i.name !="b":
                        if i.name == 'h3':
                            ## creating a personal split symbol "--;" for convenience
                            file.write('{}--;'.format(rank))
                            id_name=str(i.contents)
                            file.write('{}--;'.format(re.search("http.+[0-9]+\"",id_name).group(0)[:-1]))
                            file.write('{}--;'.format(re.search("\>.+\<",id_name).group(0)[1:-1]))
                        else:
                            file.write('{}--;'.format(i.text))
                for j in list(info.next_siblings):
                    for k in j.contents:
                        file.write('{}--;'.format(k.text))
                file.write('\n')
                # increase the rank and go to next moive
                rank+=1
        file.flush()
## notice that sales in china is in CNY¥ while its USD$ for global & NA

In [7]:
## see if crutial information has been scraped 
for area in ["china","global","NorthAmerica"]:
    with open('mtime_data_'+area+'.txt', mode='r') as file:
        contents=file.read()
    assert len(re.findall("http",contents))==100
    ## check scraped information for a movie
    basic_info=[]
    info_size=[]
    with open('mtime_data_'+area+'.txt', mode='r') as file:
        for line in file:
            basic=line.strip("\n--;").split("--;")
            basic_info.append(basic)
            info_size.append(len(basic))
    # output a sample basic information
    for i in basic_info:
        if len(i)==max(info_size):
            output=i
        break
    print(output)

['1', 'http://movie.mtime.com/229733', '战狼2', 'Wolf Warriors Ⅱ', '首日1.02亿|首周9.97亿|连冠4周', '2017年07月27日上映\xa02D/3D/IMAX', '导演：吴京', '主演：吴京/弗兰克·格里罗', '发行公司：北京京西文化旅游股份有限公司...', '7.4', '17009人评分', '56.82亿', '累计人次：1.60亿']
['1', 'http://movie.mtime.com/45997', '阿凡达', 'Avatar', '2009年12月17日英国上映\xa02D/3D/IMAX', '导演：詹姆斯·卡梅隆', '主演：萨姆·沃辛顿/佐伊·索尔达娜', '发行公司：中影...', '8.8', '68014人评分', '27.88亿']
['1', 'http://movie.mtime.com/192895', '星球大战：原力觉醒', 'Star Wars: The Force Awakens', '首日1.19亿|首周3.91亿', '2015年12月18日美国上映\xa02D/3D/IMAX3D', '导演：J·J·艾布拉姆斯', '主演：黛茜·雷德利/约翰·波耶加', '发行公司：中影...', '7.6', '9137人评分', '9.37亿']
